<a href="https://colab.research.google.com/github/Shakermaker001/Home-Sales/blob/main/Home_Sales_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:7 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [50.4 kB]
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,346 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main amd6

In [ ]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [ ]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
homes_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)

In [ ]:
# 2. Create a temporary view of the DataFrame.
homes_df.createOrReplaceTempView("homes")
spark.sql("SELECT * FROM homes").show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [ ]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
spark.sql("""
select ROUND(AVG(price),2) AS average_price, date_built
from homes
where bedrooms = "4"
group by date_built
order by date_built desc
""").show()


+-------------+----------+
|average_price|date_built|
+-------------+----------+
|    296576.69|      2017|
|    296050.24|      2016|
|    307908.86|      2015|
|    299073.89|      2014|
|    299999.39|      2013|
|    298233.42|      2012|
|     302141.9|      2011|
|    296800.75|      2010|
+-------------+----------+



In [ ]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
spark.sql("""
select ROUND(AVG(price),2) AS average_price, date_built
from homes
where bedrooms = "3"
  AND bathrooms = "3"
group by date_built
order by date_built desc
""").show()


+-------------+----------+
|average_price|date_built|
+-------------+----------+
|    292676.79|      2017|
|    290555.07|      2016|
|     288770.3|      2015|
|    290852.27|      2014|
|    295962.27|      2013|
|    293683.19|      2012|
|    291117.47|      2011|
|    292859.62|      2010|
+-------------+----------+



In [ ]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
spark.sql("""
select ROUND(AVG(price),2) AS average_price, date_built
from homes
where bedrooms = "3"
  AND bathrooms = "3"
  AND floors = "2"
  AND sqft_living >= "2000"
group by date_built
order by date_built desc
""").show()


+-------------+----------+
|average_price|date_built|
+-------------+----------+
|    280317.58|      2017|
|     293965.1|      2016|
|    297609.97|      2015|
|    298264.72|      2014|
|    303676.79|      2013|
|    307539.97|      2012|
|    276553.81|      2011|
|    285010.22|      2010|
+-------------+----------+



In [ ]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()
spark.sql("""
select ROUND(AVG(price),2) AS average_price, view
from homes
group by view
having avg(price) >= "350000"
order by view desc
""").show()


print("--- %s seconds ---" % (time.time() - start_time))

+-------------+----+
|average_price|view|
+-------------+----+
|   1061201.42|  99|
|   1053739.33|  98|
|   1129040.15|  97|
|   1017815.92|  96|
|    1054325.6|  95|
|    1033536.2|  94|
|   1026006.06|  93|
|    970402.55|  92|
|   1137372.73|  91|
|   1062654.16|  90|
|   1107839.15|  89|
|   1031719.35|  88|
|    1072285.2|  87|
|   1070444.25|  86|
|   1056336.74|  85|
|   1117233.13|  84|
|   1033965.93|  83|
|    1063498.0|  82|
|   1053472.79|  81|
|    991767.38|  80|
+-------------+----+
only showing top 20 rows

--- 1.1537656784057617 seconds ---


In [ ]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table homes")

DataFrame[]

In [ ]:
# 8. Check if the table is cached.
spark.catalog.isCached('homes')

True

In [ ]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

spark.sql("""
select ROUND(AVG(price),2) AS average_price, view
from homes
group by view
having avg(price) >= "350000"
order by view desc
""").show()


print("--- %s seconds ---" % (time.time() - start_time))


+-------------+----+
|average_price|view|
+-------------+----+
|   1061201.42|  99|
|   1053739.33|  98|
|   1129040.15|  97|
|   1017815.92|  96|
|    1054325.6|  95|
|    1033536.2|  94|
|   1026006.06|  93|
|    970402.55|  92|
|   1137372.73|  91|
|   1062654.16|  90|
|   1107839.15|  89|
|   1031719.35|  88|
|    1072285.2|  87|
|   1070444.25|  86|
|   1056336.74|  85|
|   1117233.13|  84|
|   1033965.93|  83|
|    1063498.0|  82|
|   1053472.79|  81|
|    991767.38|  80|
+-------------+----+
only showing top 20 rows

--- 0.6393570899963379 seconds ---


In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
homes_df.write.parquet('parquet_delay_basic', mode='overwrite')
homes_df.write.partitionBy("date_built").mode("overwrite").parquet("homes_partitioned")

In [ ]:
# 11. Read the parquet formatted data.
homes_df = spark.read.parquet("homes_partitioned")

In [ ]:
# 12. Create a temporary table for the parquet data.
homes_df.createOrReplaceTempView("homes")

In [ ]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

spark.sql("""
select ROUND(AVG(price),2) AS average_price, view
from homes
group by view
having avg(price) >= "350000"
order by view desc
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+-------------+----+
|average_price|view|
+-------------+----+
|   1061201.42|  99|
|   1053739.33|  98|
|   1129040.15|  97|
|   1017815.92|  96|
|    1054325.6|  95|
|    1033536.2|  94|
|   1026006.06|  93|
|    970402.55|  92|
|   1137372.73|  91|
|   1062654.16|  90|
|   1107839.15|  89|
|   1031719.35|  88|
|    1072285.2|  87|
|   1070444.25|  86|
|   1056336.74|  85|
|   1117233.13|  84|
|   1033965.93|  83|
|    1063498.0|  82|
|   1053472.79|  81|
|    991767.38|  80|
+-------------+----+
only showing top 20 rows

--- 1.463167667388916 seconds ---


In [ ]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table homes")

DataFrame[]

In [ ]:
# 15. Check if the home_sales is no longer cached
if spark.catalog.isCached("homes"):
  print("there is cache")
else:
  print("no cache money")


no cache money
